In [ ]:
import cv2
import face_recognition
import os
import pymysql
import pandas as pd
import datetime
import numpy as np
import serial
import time

path = 'C:/Users/cyoun/pps/Pictures/Camera Roll/pass'
images = []
classNames = []
myList = os.listdir(path)
ser = serial.Serial(port='COM5',
                    baudrate=115200,
)


for cl in myList:
    curImg = face_recognition.load_image_file(f'{path}/{cl}')
    images.append(curImg)
    classNames.append(os.path.splitext(cl)[0])
print(classNames)



def find_encodings(image_paths):
    encode_list = []
    for image_path in image_paths:
        encode = face_recognition.face_encodings(image_path)[0]
        encode_list.append(encode)
    return encode_list


encodeListKnown = find_encodings(images)
print(f'Encoding Complete {len(encodeListKnown)}')


cap = cv2.VideoCapture(0)

while True:
    success, cap_img = cap.read()
    imgS = cv2.resize(cap_img, (0, 0), None, 0.25, 0.25)
    imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)
    now = datetime.datetime.now()
    reg1 = datetime.datetime(now.year,now.month,now.day,9,0,0,0)
    reg2 = datetime.datetime(now.year,now.month,now.day,18,0,0,0)
  
    faceCurFrame = face_recognition.face_locations(imgS)
    encodesCurFrame = face_recognition.face_encodings(imgS, faceCurFrame)


    for encodeFace, faceLoc in zip(encodesCurFrame, faceCurFrame):
        matches = face_recognition.compare_faces(encodeListKnown, encodeFace)
        faceDis = face_recognition.face_distance(encodeListKnown, encodeFace)
        

        matchIndex = np.argmin(faceDis)

        if matches[matchIndex]:
            ser.write(b'a')
            name = classNames[matchIndex].upper()
            data = (now,name)
            
            y1, x2, y2, x1 = faceLoc
            y1, x2, y2, x1 = y1*4, x2*4, y2*4, x1*4
            cv2.rectangle(cap_img, (x1, y1), (x2, y2), (0,0,255),2)
            cv2.rectangle(cap_img, (x1, y2-35), (x2, y2), (0,0,255), cv2.FILLED)
            cv2.putText(cap_img, name, (x1+6, y2-6), cv2.FONT_HERSHEY_COMPLEX, 1, (255,255,255), 2)
           
          
        else:
            name = '???'
            ser.write(b'c')
            
            y1, x2, y2, x1 = faceLoc
            y1, x2, y2, x1 = y1 * 4, x2 * 4, y2 * 4, x1 * 4
            cv2.rectangle(cap_img, (x1, y1), (x2, y2), (0, 0, 255), 2)
            cv2.rectangle(cap_img, (x1, y2 - 35), (x2, y2), (0, 0, 255), cv2.FILLED)
            cv2.putText(cap_img, name, (x1 + 6, y2 - 6), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)
            
            
    
        if  now <reg1:
                    sql = '''UPDATE `Employ`
                             SET emp_reg_in = %s, emp_reg_out = 'null', emp_reg_check = '1'
                             WHERE emp_id = %s ;'''
                    cursor.execute(sql,data)
                    db.commit()
                  

        if now > reg2:
                    sql = '''UPDATE `Employ`
                              SET emp_reg_out = %s, emp_reg_check = '0'
                              WHERE emp_id = %s ;'''
                    cursor.execute(sql,data)
                    db.commit()
                    count = 0

            
    
                
    cv2.imshow('cam', cap_img)
    cv2.waitKey(1)

['1234', '201662071', '201762039', '201762064']
Encoding Complete 4


In [ ]:
import pymysql
import pandas as pd

db = pymysql.connect(
    host='127.0.0.1',
    port=3306,
    user='root',
    passwd='cy1156732!', 
    charset='utf8',
    db = 'employ_mange'
)

cursor = db.cursor()
cursors = db.cursor()
